In [146]:
import os                              
import numpy as np
from numpy import number                    
import pandas as pd                     
import matplotlib.pyplot as plt  
import math
from typing import List
from azure.quantum.optimization import Problem, ProblemType, Term
from azure.quantum.optimization import ParallelTempering
from azure.quantum.optimization import Term, SlcTerm

<h3 style="color:blue;"> Data Preprocessing </h3>

In [147]:
stocks = [stock.split('.')[0] for stock in sorted(os.listdir("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)"))]
stocks = stocks[1:]

print(stocks)

['ADANIENT', 'ADANIGREEN', 'ADANIPORTS', 'ADANIPOWER', 'ADANITRANS', 'ASIANPAINT', 'ATGL', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BHARTIARTL', 'BRITANNIA', 'COALINDIA', 'DABUR', 'DIVISLAB', 'DMART', 'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HINDUNILVR', 'HINDZINC', 'ICICIBANK', 'INFY', 'IOC', 'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'M&M', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PIDILITIND', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SBIN', 'SIEMENS', 'SUNPHARMA', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'WIPRO']


In [148]:
dates = pd.date_range('2020-06-27', '2022-05-27') 
data = pd.DataFrame({'Time': dates})


In [156]:
for stock in stocks:
    prices = pd.read_csv("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)/"+stock+".csv", usecols=['Date', 'WAP'])
    #print(prices)
    prices['Date'] = pd.to_datetime(prices['Date'], dayfirst = True)  
    #print(prices)
    prices.rename(                                                            
        columns={"Date": "Time", "WAP": stock},
        inplace=True
    )
    data = pd.merge(data,prices)


In [157]:
cp = data .drop(['Time'], axis=1).tail(1).to_numpy()
cp = cp[0]

In [158]:
# Calculating daily returns for each stock

r = data[(data['Time'] >= '2020-06-27')] \
    .drop(['Time'], axis=1) \
    .pct_change(fill_method='ffill')


In [159]:
# Calculating mean and covariance matrix for each stocks
mu = r.mean().to_numpy()
sigma = r.cov().to_numpy()
n=len(stocks)

In [160]:
'''
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "e87e6b08-4913-4522-9206-ba18a56ee0fa",
    resource_group = "AzureQuantum",
    name = "MTCProjectQuantumSharique",
    location = "East US"
)


from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "6e02aef9-2670-4418-92b8-7659cf5605d2",
    resource_group = "azurequantum",
    name = "workspace1",
    location = "japaneast"
)
'''

from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "ad18d2e0-288d-409a-bc11-5231714f4fb8",
    resource_group = "rg-mtc-poc-quantum",
    name = "ws-mtc-poc-quantum",
    location = "japaneast"
)



<h3 style="color:blue;">Optimized Portfolio </h3>

In [161]:
import functions

In [162]:

t1= functions.return_terms(mu,n)
t2= functions.risk(sigma, n, 1)
terms=[]
terms=t1+t2
problem = Problem(name="Portfolio optimization", problem_type=ProblemType.pubo, terms=terms)
solver = ParallelTempering(workspace)
solution = solver.optimize(problem)


Number of terms in return:  49
Number of terms in risk:  2401
....

In [163]:
print(solution['configuration'])
result = solution['configuration']
#print(type(result))

selected_stocks=[]
prices_selected=[]
index_prices_selected= []

daily_prices = data.drop(['Time'], axis=1).to_numpy()
#print(daily_prices)

for i in result.keys():
    if result[i]:
        selected_stocks.append(stocks[int(i)])
        prices_selected.append(cp[int(i)])
        index_prices_selected.append(int(i))
        
print("\nSelected stocks are -\n", selected_stocks)

print("\nPrice of selected stocks are -\n", prices_selected)

{'0': 1, '1': 1, '2': 0, '3': 1, '4': 1, '5': 0, '6': 1, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 0, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 1, '25': 0, '26': 0, '27': 1, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 0, '38': 0, '39': 1, '40': 1, '41': 1, '42': 1, '43': 1, '44': 0, '45': 1, '46': 1, '47': 0, '48': 1}

Selected stocks are -
 ['ADANIENT', 'ADANIGREEN', 'ADANIPOWER', 'ADANITRANS', 'ATGL', 'INFY', 'JSWSTEEL', 'SBIN', 'SIEMENS', 'SUNPHARMA', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'TITAN', 'WIPRO']

Price of selected stocks are -
 [2070.0593123381705, 2175.5634118259386, 324.9441561175404, 2148.1660402684565, 2418.4302410036316, 1455.220770671192, 548.9821474924378, 470.32481588044175, 2367.7135457135455, 901.0188212463532, 429.0204163091215, 1047.8110535645603, 1114.4954922105685, 2140.019082601684, 463.4882399473741]


In [164]:
Budget = 1000000
risk_aversion_index = 1
daily_returns = functions.find_expected_daily_return(mu,selected_stocks,index_prices_selected)
annual_returns_percent = round((daily_returns*252 *100),3)
annual_returns = round(Budget+ (daily_returns*252*Budget),1)
risk_percent = round(functions.find_risk(selected_stocks,n,result,sigma)*(252**0.5),4)

In [165]:
# Printing results
print("OPTIMIZSED PORTFOLIO :\n")
no_of_shares = functions.distributed_budget(1000000, arr= selected_stocks, cp=cp, index_prices_selected= index_prices_selected)
print("Expected Annual Return on Investment : ", annual_returns_percent, "%")
print("\nConstraints:")
print("\nBudget : ",  u"\u20B9", Budget)
print("Risk Aversion index of :" , risk_aversion_index,"\n")
print("Expected Annual Return: ",  u"\u20B9", annual_returns)
print("Annual Risk percentage: ", risk_percent ,"%")

OPTIMIZSED PORTFOLIO :

ADANIENT  :  32  shares
ADANIGREEN  :  30  shares
ADANIPOWER  :  205  shares
ADANITRANS  :  31  shares
ATGL  :  27  shares
INFY  :  45  shares
JSWSTEEL  :  121  shares
SBIN  :  141  shares
SIEMENS  :  28  shares
SUNPHARMA  :  73  shares
TATAMOTORS  :  155  shares
TATASTEEL  :  63  shares
TECHM  :  59  shares
TITAN  :  31  shares
WIPRO  :  143  shares
Expected Annual Return on Investment :  79.036 %

Constraints:

Budget :  ₹ 1000000
Risk Aversion index of : 1 

Expected Annual Return:  ₹ 1790362.3
Annual Risk percentage:  21.3926 %


<h3 style="color:blue;"> Diversified Portfolio </h3>
Selecting a portfolio after applying a budget constraint which restricts number
of stocks.

In [166]:
terms=[]
t1 = functions.return_terms(mu,n)
t2 = functions.risk(sigma,n,1)
t3 = functions.budget_constraint(2*n,n,15)
terms= t1+ t2 + t3
problem = Problem(name="Portfolio optimization", problem_type=ProblemType.pubo, terms=terms)
solver = ParallelTempering(workspace)
solution = solver.optimize(problem)

Number of terms in return:  49
Number of terms in risk:  2401
.......

In [167]:
print(solution['configuration'])
result = solution['configuration']

selected_stocks=[]
prices_selected=[]
index_prices_selected= []

daily_prices = data.drop(['Time'], axis=1).to_numpy()


for i in result.keys():
    if result[i]:
        selected_stocks.append(stocks[int(i)])
        prices_selected.append(cp[int(i)])
        index_prices_selected.append(int(i))
        
print("\nSelected stocks are -\n", selected_stocks)

print("\nPrice of selected stocks are -\n", index_prices_selected)

{'0': 1, '1': 1, '2': 0, '3': 1, '4': 1, '5': 1, '6': 1, '7': 0, '8': 0, '9': 0, '10': 0, '11': 0, '12': 0, '13': 0, '14': 0, '15': 0, '16': 0, '17': 1, '18': 0, '19': 0, '20': 0, '21': 0, '22': 0, '23': 0, '24': 0, '25': 0, '26': 0, '27': 0, '28': 0, '29': 0, '30': 0, '31': 0, '32': 0, '33': 0, '34': 0, '35': 0, '36': 0, '37': 1, '38': 1, '39': 1, '40': 0, '41': 1, '42': 1, '43': 1, '44': 0, '45': 1, '46': 0, '47': 0, '48': 1}

Selected stocks are -
 ['ADANIENT', 'ADANIGREEN', 'ADANIPOWER', 'ADANITRANS', 'ASIANPAINT', 'ATGL', 'HCLTECH', 'RELIANCE', 'SBILIFE', 'SBIN', 'SUNPHARMA', 'TATAMOTORS', 'TATASTEEL', 'TECHM', 'WIPRO']

Price of selected stocks are -
 [0, 1, 3, 4, 5, 6, 17, 37, 38, 39, 41, 42, 43, 45, 48]


In [168]:
Budget = 1000000
risk_aversion_index = 1
stock_appetite = 15
daily_returns = functions.find_expected_daily_return(mu,selected_stocks,index_prices_selected)
annual_returns_percent = round((daily_returns*252 *100),3)
annual_returns = round(Budget+ (daily_returns*252*Budget),1)
risk_percent = round(functions.find_risk(selected_stocks,n,result,sigma)*(252**0.5),4)


In [169]:
# Printing results

print("\n DIVERSIFIED PORTFOLIO:\n")
no_of_shares = functions.distributed_budget(1000000, selected_stocks, cp, index_prices_selected)

print("\nConstraints:")
print("\nBudget : ",  u"\u20B9", Budget)
print("Stock Appetite: ", stock_appetite , " stocks")
print("Risk Aversion index of :" , 1,"\n")
print("Expected Annual Return on Investment : ", annual_returns_percent, "%")
print("Expected Annual Return: ",  u"\u20B9", annual_returns)
print("Annual Risk percentage: ", risk_percent ,"%")



 DIVERSIFIED PORTFOLIO:

ADANIENT  :  32  shares
ADANIGREEN  :  30  shares
ADANIPOWER  :  205  shares
ADANITRANS  :  31  shares
ASIANPAINT  :  23  shares
ATGL  :  27  shares
HCLTECH  :  66  shares
RELIANCE  :  25  shares
SBILIFE  :  59  shares
SBIN  :  141  shares
SUNPHARMA  :  73  shares
TATAMOTORS  :  155  shares
TATASTEEL  :  63  shares
TECHM  :  59  shares
WIPRO  :  143  shares

Constraints:

Budget :  ₹ 1000000
Stock Appetite:  15  stocks
Risk Aversion index of : 1 

Expected Annual Return on Investment :  73.644 %
Expected Annual Return:  ₹ 1736440.8
Annual Risk percentage:  20.5064 %


<h3 style="color:blue;"> Validation </h3>
Verification of the predicted returns with the actual returns of the coming month

In [170]:
dates = pd.date_range('2022-05-27', '2022-06-27')
data_actual=  pd.DataFrame({'Time': dates})


In [171]:
def predicting_performance(data_actual):

    for stock in stocks:
        prices = pd.read_csv("Datasets/Bombay Stock Exchange Top 50 (last 20 years data)/"+stock+".csv", usecols=['Date', 'WAP'])
        prices['Date'] = pd.to_datetime(prices['Date'], dayfirst = True)  
        prices.rename(                                                            
            columns={"Date": "Time", "WAP": stock},
            inplace=True
        )
        data_actual = pd.merge(data_actual,prices)
    return data_actual
data_actual=predicting_performance(data_actual)

In [172]:
cp_actual = data_actual .drop(['Time'], axis=1).tail(1).to_numpy()
cp_actual = cp_actual[0]

In [173]:
market_price=[]
for i in index_prices_selected:
    market_price.append(cp_actual[i])

In [174]:

current_value_of_investment=1000000
for i in range(len(market_price)):
    current_value_of_investment = current_value_of_investment + (market_price[i] * no_of_shares[i])

print("\nActual Monthly Return on Investment: ", (((current_value_of_investment-1000000)/1000000)*100) /12, "%")

print("\nPredicted Monthly Returns on investment: ", daily_returns*22*100, "%")


Actual Monthly Return on Investment:  7.795599520559737 %

Predicted Monthly Returns on investment:  6.4292448542798875 %


In [175]:
print("\nPredicted Monthly Returns on investment: ", (((1+ daily_returns)**252)-1)*100/12, "%")


Predicted Monthly Returns on investment:  9.052058798084547 %
